In [3]:
import sys
import csv
import pandas as pd
import tweepy
import string
from string import maketrans 
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, KFold

In [12]:
import pandas as pd
from pandas_summary import DataFrameSummary
from sklearn.externals import joblib

In [5]:
wordnet_lemmatizer = WordNetLemmatizer()

In [6]:
df_train = pd.read_csv('Train_rev1.csv')

In [7]:
df_train = df_train[['FullDescription', 'SalaryNormalized']]
df_train.columns = [['message', 'salary']]

In [8]:
df_train = df_train.reset_index(drop = True)

In [9]:
def text_process(s):
    s = str(s)
    s = s.translate(None, string.punctuation)
    s = s.translate(None, '1234567890')
    s = [word for word in s.split() if not (word.startswith('http'))]
    s = ' '.join(s)
    s = re.sub(' +',' ',s)
    s = s.decode('ascii', 'ignore')
    s = s.lower()
    s = [word for word in s.split() if word not\
         in stopwords.words('english') + ['k']]
    s = [wordnet_lemmatizer.lemmatize(word) for word in s]
    sentence = ' '.join(s)
    return sentence

In [10]:
messages = df_train['message'].values
n_message = len(messages)
token_values = np.zeros(n_message).astype(str)
for idx in tqdm(range(n_message)):
    token_values[idx] = text_process(messages[idx])

  0%|          | 83/244768 [00:04<3:51:01, 17.65it/s] 


KeyboardInterrupt: 

In [75]:
from sklearn.externals import joblib
joblib.dump(token_values, 'token_values.pkl')

['token_values.pkl']

In [13]:
token_values = joblib.load('token_values.pkl')

In [14]:
token_values

array(['engineering system analyst dorki',
       'stress engineer glasgow salary c',
       'mathematical modeller simulation', ...,
       'position qualified teacher subje',
       'position qualified teacher subje',
       'entrepreneurial growing private '], dtype='|S32')

In [24]:
Y_train = df_train['salary'].values.astype(float).reshape(-1)

In [16]:
kfold = KFold(n_splits = 3, shuffle = True, random_state=1234)

In [30]:
from sklearn.linear_model import LassoCV, ElasticNetCV
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor

In [31]:
from sklearn.metrics import make_scorer

def mape_loss_func(labels, preds):
    err = 100. * np.mean(np.abs(1. - np.true_divide(preds, labels))) 
    return err

mape_loss  = make_scorer(mape_loss_func, 
                          greater_is_better=False)

def mape_xgboost(preds, dtrain):
    labels = dtrain.get_label()
    err =  100. * np.mean(np.abs(1. - np.true_divide(preds, labels))) 
    return 'error', err

# Count

In [32]:
count_vectorizer = CountVectorizer()
X_train = count_vectorizer.fit_transform(token_values)

In [33]:
print X_train.shape
print Y_train.shape

(244768, 31444)
(244768,)


In [ ]:
lasso_count_model = LassoCV()
lasso_count_score = cross_val_score(lasso_count_model,
                                  X_train, Y_train,
                                  scoring = mape_loss,
                                  n_jobs=-1).mean()
print lasso_count_score